In [15]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
train = pd.read_csv('/content/drive/MyDrive/RB Dataset/train.csv')
train.head(3)

,doj,srcid,destid,final_seatcount
0,2023-03-01,45,46,2838.0
1,2023-03-01,46,45,2298.0
2,2023-03-01,45,47,2720.0


In [3]:
test = pd.read_csv('/content/drive/MyDrive/RB Dataset/test_8gqdJqH.csv')
test.head(3)

,route_key,doj,srcid,destid
0,2025-02-11_46_45,2025-02-11,46,45
1,2025-01-20_17_23,2025-01-20,17,23
2,2025-01-08_02_14,2025-01-08,2,14


In [4]:
transactions = pd.read_csv('/content/drive/MyDrive/RB Dataset/transactions.csv')
transactions.head(3)

,doj,doi,srcid,destid,srcid_region,destid_region,srcid_tier,destid_tier,cumsum_seatcount,cumsum_searchcount,dbd
0,2023-03-01,2023-01-30,45,46,Karnataka,Tamil Nadu,Tier 1,Tier 1,8.0,76.0,30.0
1,2023-03-01,2023-01-30,46,45,Tamil Nadu,Karnataka,Tier 1,Tier 1,8.0,70.0,30.0
2,2023-03-01,2023-01-30,45,47,Karnataka,Andhra Pradesh,Tier 1,Tier 1,4.0,142.0,30.0


In [5]:
# aggregate transactions
agg_txn = transactions.groupby(['doj', 'srcid', 'destid']).agg({
    'cumsum_seatcount': 'mean',
    'cumsum_searchcount': 'mean',
    'dbd': ['min', 'max', 'mean'],
    'srcid_region': 'first',
    'destid_region': 'first',
    'srcid_tier': 'first',
    'destid_tier': 'first'
}).reset_index()


agg_txn.columns = ['doj', 'srcid', 'destid', 'avg_seatcount', 'avg_searchcount',
                    'min_dbd', 'max_dbd', 'mean_dbd', 'srcid_region', 'destid_region',
                   'srcid_tier', 'destid_tier']

In [6]:
# merge features into train & test
train_merged = train.merge(agg_txn, on = ['doj', 'srcid', 'destid'], how = 'left')
test_merged = test.merge(agg_txn, on = ['doj', 'srcid', 'destid'], how = 'left')

In [7]:
train_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67200 entries, 0 to 67199
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   doj              67200 non-null  object 
 1   srcid            67200 non-null  int64  
 2   destid           67200 non-null  int64  
 3   final_seatcount  67200 non-null  float64
 4   avg_seatcount    3400 non-null   float64
 5   avg_searchcount  3400 non-null   float64
 6   min_dbd          3400 non-null   float64
 7   max_dbd          3400 non-null   float64
 8   mean_dbd         3400 non-null   float64
 9   srcid_region     3400 non-null   object 
 10  destid_region    3400 non-null   object 
 11  srcid_tier       3400 non-null   object 
 12  destid_tier      3400 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 6.7+ MB


In [8]:
train_merged.isna().sum()

,0
doj,0
srcid,0
destid,0
final_seatcount,0
avg_seatcount,63800
avg_searchcount,63800
min_dbd,63800
max_dbd,63800
mean_dbd,63800
srcid_region,63800


In [9]:
test_merged.isna().sum()

,0
route_key,0
doj,0
srcid,0
destid,0
avg_seatcount,5900
avg_searchcount,5900
min_dbd,5900
max_dbd,5900
mean_dbd,5900
srcid_region,5900


In [10]:
train_merged.duplicated().sum()

np.int64(0)

In [11]:
test_merged.duplicated().sum()

np.int64(0)

In [12]:
# list of numeric columns
num_cols = train_merged.select_dtypes(include = ['float64', 'int64']).columns.difference(['final_seatcount'])

# fill numeric columns with -1 to indicate missing behaviour
train_merged[num_cols] = train_merged[num_cols].fillna(-1)
test_merged[num_cols] = test_merged[num_cols].fillna(-1)

#list of categorical columns
categorical_cols = ['srcid_region', 'destid_region', 'srcid_tier', 'destid_tier']

# fill missing categories with a placeholder
train_merged[categorical_cols] = train_merged[categorical_cols].fillna('missing')
test_merged[categorical_cols] = test_merged[categorical_cols].fillna('missing')


In [13]:
# feature engineering
for df in [train_merged,test_merged]:
  df['doj'] = pd.to_datetime(df['doj'])
  df['dayofweek'] = df['doj'].dt.dayofweek
  df['month'] = df['doj'].dt.month

In [14]:
# feature target setup
X = train_merged.drop(['doj', 'final_seatcount'], axis=1)
y = train_merged['final_seatcount']

# one-hot encode categorical features
X = pd.get_dummies(X)
test_features = pd.get_dummies(test_merged.drop(['doj'], axis = 1))

# align columns to make sure train test have same features
X, test_features = X.align(test_features, join = 'left', axis = 1, fill_value = 0)

# train validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [17]:
# hyperparameter tuning for model
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [4, 6, 8],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}

xgb = XGBRegressor(random_state = 42, n_jobs = -1)

grid_search = GridSearchCV(
    estimator = xgb,
    param_grid = param_grid,
    scoring = 'neg_mean_squared_error',
    cv = 3,
    verbose = 1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

Fitting 3 folds for each of 48 candidates, totalling 144 fits


In [18]:
y_test_pred = best_model.predict(X_test)

print("Performance Metrics:")
mse = mean_squared_error(y_test, y_test_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)

print(f'XG Boost Performance on Validation Set:')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'R-squared (R2): {r2:.4f}')

Performance Metrics:
XG Boost Performance on Validation Set:
Mean Squared Error (MSE): 435854.9727
Root Mean Squared Error (RMSE): 660.1931
Mean Absolute Error (MAE): 429.0709
R-squared (R2): 0.6973


In [19]:
# data prediction
best_model.fit(X, y)
test_preds = best_model.predict(test_features)

In [20]:
# create submission file
submission = pd.DataFrame({
    'route_key': test_merged['route_key'],
    'final_seatcount': test_preds.round().astype(int)
})

submission.to_csv('submission_1.csv', index = False)